In [1]:
# Define a function that generates samples approximate RGO. The target is defined in Potential class.
import numpy as np
import matplotlib.pyplot as plt
import targets
import cProfile
import pstats
import random
import argparse
import os
from utils import target_func, mixing_time, TV_estimation, target_funnel
import matplotlib.ticker as ticker



In [2]:

# Define a function as the original approximate RGO
def generate_samples(step_size, x_y, f):
    dimension = f.dimension
    ite = 0
    while True:
        samples = np.random.multivariate_normal(mean = x_y, cov = step_size * np.identity(dimension), size = 2)
        # Compute the acceptance probability
        gradient = f.firstOrder(x_y)
        a = f.zeroOrder(samples[0,:])-np.dot(gradient,samples[0,:])
        b = f.zeroOrder(samples[1,:])-np.dot(gradient,samples[1,:])
        # The code works even when rho is inf. One can also take the log transformation
        rho = np.exp(b-a)
        u = np.random.uniform(0,1)
        ite = ite + 1
        if u < rho/2:
            break
    return samples[0,:],ite

# Define a function that estimates the local step size
def estimate_step_size(step_size, tolerance, y, f, size = 100, reduce = 0.5):
    dimension = f.dimension
    # Compute the desired subexponential parameter
    x_y = f.solve1(y, step_size)
    testFunction = lambda C : np.mean(np.exp(np.abs(Y)**(2/(1+f.alpha))/C))-2
    while True:
        # Generate random samples from a Gaussian distribution: \exp^{-(x-x_y)^2/(2\step_size)}
        samples = np.random.multivariate_normal(mean = x_y, cov = step_size * np.identity(dimension), size = size*2)
        Y = np.zeros(size)
        for i in range(size):
            gradient = f.firstOrder(x_y)
            a = f.zeroOrder(samples[i])-np.dot(gradient,samples[i])
            b = f.zeroOrder(samples[i+size])-np.dot(gradient,samples[i+size])
            Y[i] = b-a
        # Estimate the subexponential parameter of Y: find the smallest C>0 such that E[\exp^{\abs(Y)/C}] \leq 2 by binary search for smooth potentials
        # Initialize the interval
        left = 0
        right = dimension**(f.alpha/(f.alpha+1)) # The estimated upper bound of the subexponential parameter
        while testFunction(right)>0:
            left = right
            right = 2*right
        # Initialize the middle point
        mid = (left+right)/2
        # Initialize the value of the function
        f_mid = testFunction(mid)
        while abs(f_mid) > 1e-1:
            if f_mid > 0:
                left = mid
            else:
                right = mid
            mid = (left+right)/2
            f_mid =  testFunction(mid)
        if mid < 1 / ( np.log(6/tolerance) / np.log(2)  * 0.5) : 
            break
        else:
            step_size = step_size * reduce
            x_y = f.solve1(y, step_size) 
    return step_size, x_y

# Define the outer loop of proximal sampler
def proximal_sampler(initial_step_size, num_samples, num_iter, f, fixed, adjusted_size=10, tolerance=1e-2, reduce = 0.5):
    dimension = f.dimension

    samples = np.zeros([num_samples,num_iter,dimension])
    Ysamples = np.zeros([num_samples,dimension])
    rejections = np.zeros([num_samples, num_iter])
    step_sizes = np.zeros([num_samples,num_iter])

    # Initialize the samples for both fxied and adaptive versions
    samples[:,0,:] = np.random.multivariate_normal(mean = 2 + np.zeros(dimension), cov = np.identity(dimension), size = num_samples)
    for j in range(num_samples):
        Ysamples[j,:] = np.random.multivariate_normal(mean =  np.zeros(dimension), cov = np.identity(dimension), size = 1)
        if fixed == False:
            step_size, x_y = estimate_step_size(initial_step_size, tolerance, Ysamples[j,:], f, size = 100, reduce = reduce)
            step_sizes[j,0] = step_size
    
    if fixed == True:    
        print(f'fixed sampling') 
        for i in range(1,num_iter):
            for j in range(num_samples):
                x_y = f.solve1(Ysamples[j,:], initial_step_size)
                samples[j,i,:],ite = generate_samples(initial_step_size, x_y, f)
                Ysamples[j,:] = np.random.multivariate_normal(mean = samples[j,i,:], cov = initial_step_size * np.identity(dimension), size = 1) 
                rejections[j,i] = ite
            if i % 100 == 0:
                print(f"Steps:{i}")
                if f.times2 > 0:
                    print(f"Averaged optimization steps of the new one: {f.ite2/f.times2}")
                print(f"Averaged rejection steps : {np.mean(rejections[:,i])}")
        
        return samples
    
    if fixed == False:
        print(f'adpative sampling')
        for i in range(1,num_iter):
            for j in range(num_samples):
                # if i < 100 or (i >= 100 and np.random.uniform(0,1) < 0.001):
                if True:
                    dynamic_size = 100 if i < 5 else adjusted_size
                    step_size, x_y = estimate_step_size(1/reduce*step_sizes[j,i-1], tolerance, Ysamples[j,:], f, size = dynamic_size, reduce = reduce)
                else:
                    x_y = f.solve1(Ysamples[j,:], step_sizes[j,i-1])
                    step_size = step_sizes[j,i-1]
                samples[j,i,:],ite = generate_samples(step_size, x_y, f)
                Ysamples[j,:] = np.random.multivariate_normal(mean = samples[j,i,:], cov = step_size * np.identity(dimension), size = 1)  
                step_sizes[j,i] = step_size
                rejections[j,i] = ite  
            
            # statistics for the first sample
            if i % 100 == 0:
                print(f"Steps:{i}")
                print(f"Averaged_step_size:{np.mean(step_sizes[:,i])}")
                if f.times2 > 0:
                    print(f"Averaged optimization steps of the new one: {f.ite2/f.times2}")
                print(f"Averaged rejection steps : {np.mean(rejections[:,i])}")
    return samples, step_sizes
            


In [3]:
def funnel_sampler(seed, dimension, numIter, numSamples, initialStep):
    random.seed(seed)
    np.random.seed(seed)

    Target = target_funnel(dimension)
    
    # To Do: double check this formula
    hatC = (1 + Target.alpha) * (1 / Target.alpha)**(Target.alpha / (1 + Target.alpha)) * (1 / np.pi)**(2 / (1 + Target.alpha)) * 2**((-1 - 2 * Target.alpha) / (1 + Target.alpha))
    min_step_size = hatC / (120 * np.log(6 / 0.01) / np.log(2) * Target.L_alpha * np.sqrt(Target.dimension))
    print(f"min_step_size={min_step_size}")

    Xsamples_adaptive0, step_sizes0 = proximal_sampler(initialStep, numSamples, numIter, f=Target, fixed=False)
    
    Xsamples_adaptive1, step_sizes1 = proximal_sampler(initialStep, numSamples, numIter, f=Target, fixed=False, adjusted_size=50)
    Xsamples_adaptive2, step_sizes2 = proximal_sampler(initialStep, numSamples, numIter, f=Target, fixed=False, adjusted_size=100)
    
    Xsamples_adaptive3, step_sizes3 = proximal_sampler(initialStep, numSamples, numIter, f=Target, fixed=False, tolerance=0.1)
    Xsamples_adaptive4, step_sizes4 = proximal_sampler(initialStep, numSamples, numIter, f=Target, fixed=False, tolerance=0.001)
    
    Xsamples_adaptive5, step_sizes5 = proximal_sampler(initialStep, numSamples, numIter, f=Target, fixed=False, reduce=0.1)
    Xsamples_adaptive6, step_sizes6 = proximal_sampler(initialStep, numSamples, numIter, f=Target, fixed=False, reduce=0.9)


    direction = np.random.multivariate_normal(mean=np.zeros(dimension), cov=np.identity(dimension), size=1)
    direction = np.transpose(direction) / np.linalg.norm(direction, ord=2)
    
    realSamples = np.zeros([200000, dimension])
    for i in range(np.shape(realSamples)[0]):
        realSamples[i, :] = Target.samplesTarget()
    
    bin_num = 500
    histY, bin_edges_Y = np.histogram(np.matmul(realSamples, direction), bins=bin_num, density=True)

    num_iter = numIter
    number_samples_hist = int(0.2 * num_iter)
    min_number_samples_hist = int(0.1 * num_iter)
    number_hist = 100
    bin_size = int((number_samples_hist - min_number_samples_hist) / 100)
    index_hist = np.linspace(number_samples_hist, num_iter, number_hist).astype(int)

    def calculate_distances(Xsamples):
        distances_random = np.zeros(number_hist)
        projected_sample_random = np.matmul(Xsamples, direction)
        for i in range(number_hist):
            distances_random[i] = TV_estimation(projected_sample_random[0, min_number_samples_hist:index_hist[i]], histY, bin_edges_Y, bin_num)
        return 0.5 * distances_random

    results = {
        'default': calculate_distances(Xsamples_adaptive0),
        'size50': calculate_distances(Xsamples_adaptive1),
        'size100': calculate_distances(Xsamples_adaptive2),
        'tol0.1': calculate_distances(Xsamples_adaptive3),
        'tol0.001': calculate_distances(Xsamples_adaptive4),
        'reduce0.1': calculate_distances(Xsamples_adaptive5),
        'reduce0.9': calculate_distances(Xsamples_adaptive6)
    }

    return results, step_sizes0, index_hist

def run_experiments(seeds, dimension, numIter, numSamples, initialStep):
    all_results = {
        'default': [],
        'size50': [],
        'size100': [],
        'tol0.1': [],
        'tol0.001': [],
        'reduce0.1': [],
        'reduce0.9': []
    }
    all_step_sizes = []

    for seed in seeds:
        results, step_sizes, index_hist = funnel_sampler(seed, dimension, numIter, numSamples, initialStep)
        for key in results:
            all_results[key].append(results[key])
        all_step_sizes.append(step_sizes)
    
    mean_results = {}
    std_results = {}
    for key in all_results:
        mean_results[key] = np.mean(all_results[key], axis=0)
        std_results[key] = np.std(all_results[key], axis=0)
    
    mean_step_sizes = np.mean(all_step_sizes, axis=0)
    std_step_sizes = np.std(all_step_sizes, axis=0)

    np.save('ab_funnel_TV.npy', all_results)
    np.save('ab_funnel_size.npy', all_step_sizes)
    
    return mean_results, std_results, index_hist, mean_step_sizes, std_step_sizes

seeds = [i for i in range(3)]
dimension = 128
numIter = 1500
numSamples = 1
initialStep = 10

mean_results, std_results, index_hist, mean_step_sizes, std_step_sizes = run_experiments(seeds, dimension, numIter, numSamples, initialStep)

plt.figure(figsize=(5, 6))
for key in mean_results:
    line, = plt.plot(index_hist, mean_results[key])
    temp_mean = mean_results[key]
    temp_mean *= 2 if np.all(temp_mean == 0.5) else True
    temp_std = std_results[key]
    plt.errorbar(index_hist[np.arange(0, len(index_hist), 10)], temp_mean[np.arange(0, len(index_hist), 10)], 
                 yerr=2*temp_std[np.arange(0, len(index_hist), 10)], label=key, fmt='o', color = line.get_color(),markersize=4)

    
plt.xlabel('Iterations')
plt.ylabel('TV along a random direction')
plt.legend(fontsize='small', loc='lower left')
plt.savefig('ab_funnel_TV.pdf')
plt.show()
plt.close()

plt.figure(figsize=(5, 6))
mean_step = mean_step_sizes[0,:]
plt.plot(range(len(mean_step)), mean_step)
print(f'max of std size:{max(std_step_sizes[0,:])}')
plt.xlabel('Iterations')
plt.ylabel('Averaged step sizes')
plt.axhline(y=0.625, linestyle='--', label='0.625', color='r')
plt.axhline(y=1.25, linestyle='--', label='1.25', color='g')
plt.legend()
plt.savefig('ab_funnel_size.pdf')
plt.show()
plt.close()


L=14.633606609426153
min_step_size=1.2275845356826385e-06


/tmp/ipykernel_2991756/3482300543.py:24: RuntimeWarning: overflow encountered in exp
  testFunction = lambda C : np.mean(np.exp(np.abs(Y)**(2/(1+f.alpha))/C))-2


adpative sampling
Steps:100
Averaged_step_size:0.0390625
Averaged rejection steps : 1.0
Steps:200
Averaged_step_size:0.0390625
Averaged rejection steps : 2.0
Steps:300
Averaged_step_size:0.0390625
Averaged rejection steps : 1.0
Steps:400
Averaged_step_size:0.0390625
Averaged rejection steps : 2.0
Steps:500
Averaged_step_size:0.078125
Averaged rejection steps : 5.0
Steps:600
Averaged_step_size:0.078125
Averaged rejection steps : 2.0
0.00041560975384879095
Steps:700
Averaged_step_size:0.078125
Averaged rejection steps : 2.0
Steps:800
Averaged_step_size:0.078125
Averaged rejection steps : 1.0
Steps:900
Averaged_step_size:0.078125
Averaged rejection steps : 3.0
Steps:1000
Averaged_step_size:0.0390625
Averaged rejection steps : 1.0
Steps:1100
Averaged_step_size:0.01953125
Averaged rejection steps : 2.0
Steps:1200
Averaged_step_size:0.078125
Averaged rejection steps : 6.0
Steps:1300
Averaged_step_size:0.0390625
Averaged rejection steps : 1.0
Steps:1400
Averaged_step_size:0.078125
Averaged re